#### **Note.** This Jupyter notebook is designed to work standalone without any additional file. The only exception is that after you clone the github repo, you need to manually upload your pretrained models (e.g., models trained with normal or exponential data as described in README) to respective folders.

#### This notebook seems to work fine on colab.research.google.com. On your local computer, some libraries might need to be installed beforehand.

In [0]:
#clone the repository to obtain pretained models and drawing utilities
#install required packages that are not automatically installed on Colab
!git clone https://github.com/wouterkool/attention-learn-to-route.git
!pip install tensorboard_logger
%cd /content/attention-learn-to-route


Cloning into 'attention-learn-to-route'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 246 (delta 0), reused 1 (delta 0), pack-reused 243
Receiving objects: 100% (246/246), 74.66 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/attention-learn-to-route


In [0]:
!python generate_data.py --problem tsp --graph_size 20 --name uni --seed 1234


[[0.1915194503788923, 0.6221087710398319], [0.4377277390071145, 0.7853585837137692], [0.7799758081188035, 0.2725926052826416], [0.2764642551430967, 0.8018721775350193], [0.9581393536837052, 0.8759326347420947], [0.35781726995786667, 0.5009951255234587], [0.6834629351721363, 0.7127020269829002], [0.37025075479039493, 0.5611961860656249], [0.5030831653078097, 0.013768449590682241], [0.772826621612374, 0.8826411906361166], [0.3648859839013723, 0.6153961784334937], [0.07538124164297655, 0.3688240060019745], [0.9331401019825216, 0.6513781432265774], [0.3972025777261542, 0.7887301429407455], [0.31683612216887125, 0.5680986526260692], [0.8691273895612258, 0.43617342389567937], [0.8021476420801591, 0.14376682451456457], [0.7042609711183354, 0.7045813081895725], [0.21879210567408858, 0.924867628615565], [0.44214075540417663, 0.9093159589724725]]


In [0]:
%%writefile generate_data.py

import argparse
import os
import numpy as np
from utils.data_utils import check_extension, save_dataset


def generate_tsp_data(dataset_size, tsp_size):
    l = []
    for i in range(dataset_size):
        a = np.random.normal(0.5, 0.25, size=(1, tsp_size, 2))
        amin = np.min(a, axis=1)
        amax = np.max(a, axis=1)
        r = (a - amin) / (amax - amin)
        l.append(r.tolist()[0])
    return l


    #return np.random.uniform(size=(dataset_size, tsp_size, 2)).tolist()


def generate_vrp_data(dataset_size, vrp_size):
    CAPACITIES = {
        10: 20.,
        20: 30.,
        50: 40.,
        100: 50.
    }
    return list(zip(
        np.random.uniform(size=(dataset_size, 2)).tolist(),  # Depot location
        np.random.uniform(size=(dataset_size, vrp_size, 2)).tolist(),  # Node locations
        np.random.randint(1, 10, size=(dataset_size, vrp_size)).tolist(),  # Demand, uniform integer 1 ... 9
        np.full(dataset_size, CAPACITIES[vrp_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_op_data(dataset_size, op_size, prize_type='const'):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, op_size, 2))

    # Methods taken from Fischetti et al. 1998
    if prize_type == 'const':
        prize = np.ones((dataset_size, op_size))
    elif prize_type == 'unif':
        prize = (1 + np.random.randint(0, 100, size=(dataset_size, op_size))) / 100.
    else:  # Based on distance to depot
        assert prize_type == 'dist'
        prize_ = np.linalg.norm(depot[:, None, :] - loc, axis=-1)
        prize = (1 + (prize_ / prize_.max(axis=-1, keepdims=True) * 99).astype(int)) / 100.

    # Max length is approximately half of optimal TSP tour, such that half (a bit more) of the nodes can be visited
    # which is maximally difficult as this has the largest number of possibilities
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        prize.tolist(),
        np.full(dataset_size, MAX_LENGTHS[op_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_pctsp_data(dataset_size, pctsp_size, penalty_factor=3):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, pctsp_size, 2))

    # For the penalty to make sense it should be not too large (in which case all nodes will be visited) nor too small
    # so we want the objective term to be approximately equal to the length of the tour, which we estimate with half
    # of the nodes by half of the tour length (which is very rough but similar to op)
    # This means that the sum of penalties for all nodes will be approximately equal to the tour length (on average)
    # The expected total (uniform) penalty of half of the nodes (since approx half will be visited by the constraint)
    # is (n / 2) / 2 = n / 4 so divide by this means multiply by 4 / n,
    # However instead of 4 we use penalty_factor (3 works well) so we can make them larger or smaller
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }
    penalty_max = MAX_LENGTHS[pctsp_size] * (penalty_factor) / float(pctsp_size)
    penalty = np.random.uniform(size=(dataset_size, pctsp_size)) * penalty_max

    # Take uniform prizes
    # Now expectation is 0.5 so expected total prize is n / 2, we want to force to visit approximately half of the nodes
    # so the constraint will be that total prize >= (n / 2) / 2 = n / 4
    # equivalently, we divide all prizes by n / 4 and the total prize should be >= 1
    deterministic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * 4 / float(pctsp_size)

    # In the deterministic setting, the stochastic_prize is not used and the deterministic prize is known
    # In the stochastic setting, the deterministic prize is the expected prize and is known up front but the
    # stochastic prize is only revealed once the node is visited
    # Stochastic prize is between (0, 2 * expected_prize) such that E(stochastic prize) = E(deterministic_prize)
    stochastic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * deterministic_prize * 2

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        penalty.tolist(),
        deterministic_prize.tolist(),
        stochastic_prize.tolist()
    ))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--filename", help="Filename of the dataset to create (ignores datadir)")
    parser.add_argument("--data_dir", default='data', help="Create datasets in data_dir/problem (default 'data')")
    parser.add_argument("--name", type=str, required=True, help="Name to identify dataset")
    parser.add_argument("--problem", type=str, default='all',
                        help="Problem, 'tsp', 'vrp', 'pctsp' or 'op_const', 'op_unif' or 'op_dist'"
                             " or 'all' to generate all")
    parser.add_argument('--data_distribution', type=str, default='all',
                        help="Distributions to generate for problem, default 'all'.")

    parser.add_argument("--dataset_size", type=int, default=10000, help="Size of the dataset")
    parser.add_argument('--graph_sizes', type=int, nargs='+', default=[20, 50, 100],
                        help="Sizes of problem instances (default 20, 50, 100)")
    parser.add_argument("-f", action='store_true', help="Set true to overwrite")
    parser.add_argument('--seed', type=int, default=1234, help="Random seed")

    opts = parser.parse_args()

    assert opts.filename is None or (len(opts.problems) == 1 and len(opts.graph_sizes) == 1), \
        "Can only specify filename when generating a single dataset"

    distributions_per_problem = {
        'tsp': [None],
        'vrp': [None],
        'pctsp': [None],
        'op': ['const', 'unif', 'dist']
    }
    if opts.problem == 'all':
        problems = distributions_per_problem
    else:
        problems = {
            opts.problem:
                distributions_per_problem[opts.problem]
                if opts.data_distribution == 'all'
                else [opts.data_distribution]
        }

    for problem, distributions in problems.items():
        for distribution in distributions or [None]:
            for graph_size in opts.graph_sizes:

                datadir = os.path.join(opts.data_dir, problem)
                os.makedirs(datadir, exist_ok=True)

                if opts.filename is None:
                    filename = os.path.join(datadir, "{}{}{}_{}_seed{}.pkl".format(
                        problem,
                        "_{}".format(distribution) if distribution is not None else "",
                        graph_size, opts.name, opts.seed))
                else:
                    filename = check_extension(opts.filename)

                assert opts.f or not os.path.isfile(check_extension(filename)), \
                    "File already exists! Try running with -f option to overwrite."

                np.random.seed(opts.seed)
                if problem == 'tsp':
                    dataset = generate_tsp_data(opts.dataset_size, graph_size)
                elif problem == 'vrp':
                    dataset = generate_vrp_data(
                        opts.dataset_size, graph_size)
                elif problem == 'pctsp':
                    dataset = generate_pctsp_data(opts.dataset_size, graph_size)
                elif problem == "op":
                    dataset = generate_op_data(opts.dataset_size, graph_size, prize_type=distribution)
                else:
                    assert False, "Unknown problem: {}".format(problem)

                print(dataset[0])

                save_dataset(dataset, filename)

Overwriting generate_data.py


In [0]:
!python generate_data.py --problem tsp --graph_size 20 --name normal --seed 1234


[[0.5535811435958288, 0.22697231939040335], [1.0, 0.4165258371051264], [0.0, 0.6754612356500831], [0.7338412210492217, 0.3466301986824033], [0.341934043181807, 0.0], [0.8687262304609844, 0.6980747659085578], [0.7773724993204989, 0.047787457076908214], [0.17949758002833288, 0.48445728823513295], [0.5229389275233132, 0.5463898590751549], [0.9481962573909727, 0.1501581002503748], [0.24053473394527555, 0.3424335375763778], [0.4244703173257149, 0.6034393186655027], [0.9467999614959184, 0.38271803078064465], [0.6483748689629958, 0.09186238575725647], [0.24960816702459285, 0.7125392228178081], [0.14988582612994514, 0.5568234826552527], [0.8211446782654951, 0.709726980916946], [0.735758690147399, 0.4576511934778264], [0.39256182345513735, 0.41433686740879955], [0.7255220196007749, 1.0]]


In [0]:
%%writefile generate_data.py

import argparse
import os
import numpy as np
from utils.data_utils import check_extension, save_dataset


def generate_tsp_data(dataset_size, tsp_size):
    l = []
    for i in range(dataset_size):
        a = np.random.exponential(1, size=(1, tsp_size, 2))
        amin = np.min(a, axis=1)
        amax = np.max(a, axis=1)
        r = (a - amin) / (amax - amin)
        l.append(r.tolist()[0])
    return l


    #return np.random.uniform(size=(dataset_size, tsp_size, 2)).tolist()


def generate_vrp_data(dataset_size, vrp_size):
    CAPACITIES = {
        10: 20.,
        20: 30.,
        50: 40.,
        100: 50.
    }
    return list(zip(
        np.random.uniform(size=(dataset_size, 2)).tolist(),  # Depot location
        np.random.uniform(size=(dataset_size, vrp_size, 2)).tolist(),  # Node locations
        np.random.randint(1, 10, size=(dataset_size, vrp_size)).tolist(),  # Demand, uniform integer 1 ... 9
        np.full(dataset_size, CAPACITIES[vrp_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_op_data(dataset_size, op_size, prize_type='const'):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, op_size, 2))

    # Methods taken from Fischetti et al. 1998
    if prize_type == 'const':
        prize = np.ones((dataset_size, op_size))
    elif prize_type == 'unif':
        prize = (1 + np.random.randint(0, 100, size=(dataset_size, op_size))) / 100.
    else:  # Based on distance to depot
        assert prize_type == 'dist'
        prize_ = np.linalg.norm(depot[:, None, :] - loc, axis=-1)
        prize = (1 + (prize_ / prize_.max(axis=-1, keepdims=True) * 99).astype(int)) / 100.

    # Max length is approximately half of optimal TSP tour, such that half (a bit more) of the nodes can be visited
    # which is maximally difficult as this has the largest number of possibilities
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        prize.tolist(),
        np.full(dataset_size, MAX_LENGTHS[op_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_pctsp_data(dataset_size, pctsp_size, penalty_factor=3):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, pctsp_size, 2))

    # For the penalty to make sense it should be not too large (in which case all nodes will be visited) nor too small
    # so we want the objective term to be approximately equal to the length of the tour, which we estimate with half
    # of the nodes by half of the tour length (which is very rough but similar to op)
    # This means that the sum of penalties for all nodes will be approximately equal to the tour length (on average)
    # The expected total (uniform) penalty of half of the nodes (since approx half will be visited by the constraint)
    # is (n / 2) / 2 = n / 4 so divide by this means multiply by 4 / n,
    # However instead of 4 we use penalty_factor (3 works well) so we can make them larger or smaller
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }
    penalty_max = MAX_LENGTHS[pctsp_size] * (penalty_factor) / float(pctsp_size)
    penalty = np.random.uniform(size=(dataset_size, pctsp_size)) * penalty_max

    # Take uniform prizes
    # Now expectation is 0.5 so expected total prize is n / 2, we want to force to visit approximately half of the nodes
    # so the constraint will be that total prize >= (n / 2) / 2 = n / 4
    # equivalently, we divide all prizes by n / 4 and the total prize should be >= 1
    deterministic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * 4 / float(pctsp_size)

    # In the deterministic setting, the stochastic_prize is not used and the deterministic prize is known
    # In the stochastic setting, the deterministic prize is the expected prize and is known up front but the
    # stochastic prize is only revealed once the node is visited
    # Stochastic prize is between (0, 2 * expected_prize) such that E(stochastic prize) = E(deterministic_prize)
    stochastic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * deterministic_prize * 2

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        penalty.tolist(),
        deterministic_prize.tolist(),
        stochastic_prize.tolist()
    ))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--filename", help="Filename of the dataset to create (ignores datadir)")
    parser.add_argument("--data_dir", default='data', help="Create datasets in data_dir/problem (default 'data')")
    parser.add_argument("--name", type=str, required=True, help="Name to identify dataset")
    parser.add_argument("--problem", type=str, default='all',
                        help="Problem, 'tsp', 'vrp', 'pctsp' or 'op_const', 'op_unif' or 'op_dist'"
                             " or 'all' to generate all")
    parser.add_argument('--data_distribution', type=str, default='all',
                        help="Distributions to generate for problem, default 'all'.")

    parser.add_argument("--dataset_size", type=int, default=10000, help="Size of the dataset")
    parser.add_argument('--graph_sizes', type=int, nargs='+', default=[20, 50, 100],
                        help="Sizes of problem instances (default 20, 50, 100)")
    parser.add_argument("-f", action='store_true', help="Set true to overwrite")
    parser.add_argument('--seed', type=int, default=1234, help="Random seed")

    opts = parser.parse_args()

    assert opts.filename is None or (len(opts.problems) == 1 and len(opts.graph_sizes) == 1), \
        "Can only specify filename when generating a single dataset"

    distributions_per_problem = {
        'tsp': [None],
        'vrp': [None],
        'pctsp': [None],
        'op': ['const', 'unif', 'dist']
    }
    if opts.problem == 'all':
        problems = distributions_per_problem
    else:
        problems = {
            opts.problem:
                distributions_per_problem[opts.problem]
                if opts.data_distribution == 'all'
                else [opts.data_distribution]
        }

    for problem, distributions in problems.items():
        for distribution in distributions or [None]:
            for graph_size in opts.graph_sizes:

                datadir = os.path.join(opts.data_dir, problem)
                os.makedirs(datadir, exist_ok=True)

                if opts.filename is None:
                    filename = os.path.join(datadir, "{}{}{}_{}_seed{}.pkl".format(
                        problem,
                        "_{}".format(distribution) if distribution is not None else "",
                        graph_size, opts.name, opts.seed))
                else:
                    filename = check_extension(opts.filename)

                assert opts.f or not os.path.isfile(check_extension(filename)), \
                    "File already exists! Try running with -f option to overwrite."

                np.random.seed(opts.seed)
                if problem == 'tsp':
                    dataset = generate_tsp_data(opts.dataset_size, graph_size)
                elif problem == 'vrp':
                    dataset = generate_vrp_data(
                        opts.dataset_size, graph_size)
                elif problem == 'pctsp':
                    dataset = generate_pctsp_data(opts.dataset_size, graph_size)
                elif problem == "op":
                    dataset = generate_op_data(opts.dataset_size, graph_size, prize_type=distribution)
                else:
                    assert False, "Unknown problem: {}".format(problem)

                print(dataset[0])

                save_dataset(dataset, filename)

Overwriting generate_data.py


In [0]:
!python generate_data.py --problem tsp --graph_size 20 --name exp --seed 1234


[[0.04336780139995939, 0.3725899129119945], [0.16070747556027948, 0.5922857427330045], [0.4638538294775591, 0.1182318722007388], [0.07923384451627163, 0.6233799636036607], [1.0, 0.8051870382771676], [0.11777203477662448, 0.2646099618309201], [0.3463421375166296, 0.4790461654868227], [0.12408898130065805, 0.3145445007541868], [0.2006306078036347, 0.0], [0.45352244374216016, 0.8267779034514137], [0.12134819870178674, 0.3657511425229183], [0.0, 0.17334714305702045], [0.8487082869826074, 0.4039024774193534], [0.1382214616590773, 0.5984351643635087], [0.09778457977956194, 0.32070272971702574], [0.6317075161801906, 0.21717386700500294], [0.4981721509735848, 0.05490027490104686], [0.3683008324636744, 0.46821989951624987], [0.05445501470450839, 1.0], [0.16325332759297706, 0.926929627525629]]


# BENCHMARK

In [0]:
### At this point, you need to upload trained respective models to folders pretrained/tsp20_normal, pretrained/tsp20_exp

#### Please see README for directions to train models using data from normal and exponential distributions

## model trained by uniform data

In [0]:
### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp_20 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [00:01<00:00,  6.62it/s]
Average cost: 3.8469574451446533 +- 0.006181415915489197
Average serial duration: 0.14815111541748047 +- 0.0014710603475011662
Average parallel duration: 0.00014467882364988327
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp_20 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [00:01<00:00,  7.89it/s]
Average cost: 4.0230712890625 +- 0.005070921182632446
Average serial duration: 0.12489537239074706 +- 0.0003144771939559984
Average parallel duration: 0.00012196813710033893
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp_20 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [00:01<00:00,  8.54it/s]
Average cost: 3.8908984661102295 +- 0.004641308784484864
Average serial duration: 0.11471791801452637 +- 0.00013030332244559376
Average parallel duration: 0.0001120292168110609
Calculated total duration: 0:00:01


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp_20 --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10000/10000 [15:18<00:00, 10.86it/s]
Average cost: 3.8381543159484863 +- 0.006112644672393799
Average serial duration: 0.09167320513725281 +- 3.117995698802496e-05
Average parallel duration: 0.09167320513725281
Calculated total duration: 0:15:16


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp_20 --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10000/10000 [15:20<00:00, 10.86it/s]
Average cost: 4.0072712898254395 +- 0.004964374601840973
Average serial duration: 0.09189823930263519 +- 3.1766859988534636e-05
Average parallel duration: 0.09189823930263519
Calculated total duration: 0:15:18


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp_20 --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10000/10000 [15:22<00:00, 10.85it/s]
Average cost: 3.876112222671509 +- 0.004581215977668762
Average serial duration: 0.09203371646404267 +- 3.346870868492905e-05
Average parallel duration: 0.09203371646404267
Calculated total duration: 0:15:20


### beam 

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp_20 --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [12:45<00:00, 72.80s/it]
Average cost: 3.8357863426208496 +- 0.006096035838127136
Average serial duration: 76.99149408950805 +- 0.10007011806510074
Average parallel duration: 0.07518700594678521
Calculated total duration: 0:12:31


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp_20 --decode_strategy bs --width 1280


  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [12:46<00:00, 72.80s/it]
Average cost: 4.00300407409668 +- 0.004933704435825348
Average serial duration: 77.00303399429322 +- 0.10050436876072448
Average parallel duration: 0.07519827538505197
Calculated total duration: 0:12:31


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp_20 --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp_20/epoch-99.pt
100% 10/10 [12:46<00:00, 72.86s/it]
Average cost: 3.872048854827881 +- 0.004557388424873352
Average serial duration: 77.04389841041565 +- 0.10010861828241684
Average parallel duration: 0.07523818204142153
Calculated total duration: 0:12:32


## model trained by normal data

### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  7.74it/s]
Average cost: 3.8487350940704346 +- 0.006188065409660339
Average serial duration: 0.12723925552368165 +- 0.00033710665387614636
Average parallel duration: 0.00012425708547234536
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  8.51it/s]
Average cost: 4.021048069000244 +- 0.005075105428695679
Average serial duration: 0.1261111213684082 +- 0.0003147020906978365
Average parallel duration: 0.00012315539196133615
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  8.54it/s]
Average cost: 3.8872718811035156 +- 0.004643577039241791
Average serial duration: 0.11479463768005371 +- 0.00013039843674037734
Average parallel duration: 0.00011210413835942745
Calculated total duration: 0:00:01


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10000/10000 [15:15<00:00, 10.91it/s]
Average cost: 3.8381524085998535 +- 0.0061155444383621214
Average serial duration: 0.09139069859981537 +- 3.13256571533562e-05
Average parallel duration: 0.09139069859981537
Calculated total duration: 0:15:13


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10000/10000 [15:19<00:00, 10.88it/s]
Average cost: 4.006791591644287 +- 0.004963922798633575
Average serial duration: 0.09178277904987335 +- 3.0130162999989738e-05
Average parallel duration: 0.09178277904987335
Calculated total duration: 0:15:17


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10000/10000 [15:17<00:00, 10.89it/s]
Average cost: 3.8755555152893066 +- 0.004577849805355072
Average serial duration: 0.09161883289813995 +- 3.463647558383523e-05
Average parallel duration: 0.09161883289813995
Calculated total duration: 0:15:16


### beam search

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [12:43<00:00, 72.57s/it]
Average cost: 3.835773468017578 +- 0.006095810532569885
Average serial duration: 76.70787101364135 +- 0.09933484058526675
Average parallel duration: 0.07491003028675913
Calculated total duration: 0:12:29


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [12:43<00:00, 72.57s/it]
Average cost: 4.002930641174316 +- 0.0049324694275856016
Average serial duration: 76.71099352416992 +- 0.09952827258098469
Average parallel duration: 0.07491307961344719
Calculated total duration: 0:12:29


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_normal --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_normal/epoch-99.pt
100% 10/10 [12:43<00:00, 72.74s/it]
Average cost: 3.8719804286956787 +- 0.004555678069591522
Average serial duration: 76.78243901138306 +- 0.09842724740134741
Average parallel duration: 0.07498285059705377
Calculated total duration: 0:12:29


## model trained by exp data

### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  7.90it/s]
Average cost: 3.8662891387939453 +- 0.006299136877059936
Average serial duration: 0.12459056549072266 +- 0.00031113642457168624
Average parallel duration: 0.00012167047411203385
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  8.41it/s]
Average cost: 4.049468040466309 +- 0.005252897143363953
Average serial duration: 0.11675377235412597 +- 0.0001380739674404142
Average parallel duration: 0.00011401735581457614
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  8.49it/s]
Average cost: 3.8815555572509766 +- 0.004659999907016754
Average serial duration: 0.11548481101989747 +- 0.00012182340045244305
Average parallel duration: 0.00011277813576161862
Calculated total duration: 0:00:01


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10000/10000 [15:22<00:00, 10.83it/s]
Average cost: 3.8418924808502197 +- 0.006148285269737244
Average serial duration: 0.09204656994342804 +- 2.9680414589695874e-05
Average parallel duration: 0.09204656994342804
Calculated total duration: 0:15:20


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10000/10000 [15:12<00:00, 10.96it/s]
Average cost: 4.012983798980713 +- 0.005003257989883423
Average serial duration: 0.09108476479053497 +- 3.073670279938959e-05
Average parallel duration: 0.09108476479053497
Calculated total duration: 0:15:10


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10000/10000 [15:15<00:00, 10.92it/s]
Average cost: 3.8740508556365967 +- 0.004585070908069611
Average serial duration: 0.09135517461299897 +- 2.9444837636329355e-05
Average parallel duration: 0.09135517461299897
Calculated total duration: 0:15:13


### beam

In [0]:
!python -W ignore eval.py data/tsp/tsp20_uni_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [12:46<00:00, 72.89s/it]
Average cost: 3.8362414836883545 +- 0.006099634170532227
Average serial duration: 77.06917475242615 +- 0.09995013794926498
Average parallel duration: 0.07526286596916616
Calculated total duration: 0:12:32


In [0]:
!python -W ignore eval.py data/tsp/tsp20_normal_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [12:46<00:00, 72.89s/it]
Average cost: 4.004576206207275 +- 0.004940682351589203
Average serial duration: 77.0833006023407 +- 0.10003920741957335
Average parallel duration: 0.07527666074447334
Calculated total duration: 0:12:32


In [0]:
!python -W ignore eval.py data/tsp/tsp20_exp_seed1234.pkl --model pretrained/tsp20_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp20_exp/epoch-99.pt
100% 10/10 [12:46<00:00, 72.87s/it]
Average cost: 3.871830940246582 +- 0.00455826610326767
Average serial duration: 77.06826272392273 +- 0.10017743043944312
Average parallel duration: 0.07526197531633079
Calculated total duration: 0:12:32


## baselines

In [0]:
!python -W ignore -m problems.tsp.tsp_baseline farthest_insertion data/tsp/tsp20_uni_seed1234.pkl data/tsp/tsp20_normal_seed1234.pkl data/tsp/tsp20_exp_seed1234.pkl


100% 10000/10000 [00:19<00:00, 519.21it/s]
Average cost: 3.9261510540578586 +- 0.0067912789831400255
Average serial duration: 0.003496509122848511 +- 1.5766272455748915e-05
Average parallel duration: 0.0017482545614242555
Calculated total duration: 0:00:17
100% 10000/10000 [00:19<00:00, 516.81it/s]
Average cost: 4.113411405843456 +- 0.005789694990470995
Average serial duration: 0.0035043909311294557 +- 1.6315635688762028e-05
Average parallel duration: 0.0017521954655647278
Calculated total duration: 0:00:17
100% 10000/10000 [00:19<00:00, 528.39it/s]
Average cost: 3.933196804361465 +- 0.005104744878548632
Average serial duration: 0.003560198926925659 +- 1.6017586001237904e-05
Average parallel duration: 0.0017800994634628296
Calculated total duration: 0:00:17


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline nearest_insertion data/tsp/tsp20_uni_seed1234.pkl data/tsp/tsp20_normal_seed1234.pkl data/tsp/tsp20_exp_seed1234.pkl


100% 10000/10000 [00:19<00:00, 502.96it/s]
Average cost: 4.331042876579414 +- 0.007700319320583336
Average serial duration: 0.003623325824737549 +- 1.4718477127727019e-05
Average parallel duration: 0.0018116629123687744
Calculated total duration: 0:00:18
100% 10000/10000 [00:19<00:00, 503.11it/s]
Average cost: 4.4523394905958655 +- 0.006543770058541374
Average serial duration: 0.0036141402006149292 +- 1.56058280199803e-05
Average parallel duration: 0.0018070701003074646
Calculated total duration: 0:00:18
100% 10000/10000 [00:20<00:00, 499.66it/s]
Average cost: 4.3309032131195195 +- 0.006853898783231086
Average serial duration: 0.0036420297384262084 +- 1.6610718337847367e-05
Average parallel duration: 0.0018210148692131042
Calculated total duration: 0:00:18


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline random_insertion data/tsp/tsp20_uni_seed1234.pkl data/tsp/tsp20_normal_seed1234.pkl data/tsp/tsp20_exp_seed1234.pkl


100% 10000/10000 [00:12<00:00, 776.05it/s]
Average cost: 4.002858816987077 +- 0.007209318008514225
Average serial duration: 0.002333502507209778 +- 1.3590073729393837e-05
Average parallel duration: 0.001166751253604889
Calculated total duration: 0:00:11
100% 10000/10000 [00:12<00:00, 771.48it/s]
Average cost: 4.201462275425884 +- 0.006104976707380994
Average serial duration: 0.002351891899108887 +- 1.2259590146179106e-05
Average parallel duration: 0.0011759459495544434
Calculated total duration: 0:00:11
100% 10000/10000 [00:12<00:00, 772.01it/s]
Average cost: 4.029217886748669 +- 0.005922078296341571
Average serial duration: 0.002344406604766846 +- 1.178921997848706e-05
Average parallel duration: 0.001172203302383423
Calculated total duration: 0:00:11


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline nn data/tsp/tsp20_uni_seed1234.pkl data/tsp/tsp20_normal_seed1234.pkl data/tsp/tsp20_exp_seed1234.pkl


100% 10/10 [00:03<00:00,  3.25it/s]
Average cost: 4.496747266316414 +- 0.010914400457607778
Average serial duration: 0.23089101314544677 +- 0.013170191375460676
Average parallel duration: 0.00023089101314544678
Calculated total duration: 0:00:02
100% 10/10 [00:00<00:00, 11.18it/s]
Average cost: 4.768014658021927 +- 0.009324117583541715
Average serial duration: 0.011435842514038086 +- 1.6545441357673393e-06
Average parallel duration: 1.1435842514038087e-05
Calculated total duration: 0:00:00
100% 10/10 [00:00<00:00, 11.22it/s]
Average cost: 4.550673234009743 +- 0.009053436867087315
Average serial duration: 0.011440110206604005 +- 2.4947079748369583e-06
Average parallel duration: 1.1440110206604004e-05
Calculated total duration: 0:00:00


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline lkh data/tsp/tsp20_uni_seed1234.pkl data/tsp/tsp20_normal_seed1234.pkl data/tsp/tsp20_exp_seed1234.pkl


/content/attention-learn-to-route/problems/vrp/lkh/LKH-3.0.4 not found, downloading and compiling
--2019-11-01 16:23:18--  http://www.akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz
Resolving www.akira.ruc.dk (www.akira.ruc.dk)... 130.225.220.151
Connecting to www.akira.ruc.dk (www.akira.ruc.dk)|130.225.220.151|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz [following]
--2019-11-01 16:23:19--  http://akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz
Resolving akira.ruc.dk (akira.ruc.dk)... 130.225.220.230
Connecting to akira.ruc.dk (akira.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2308964 (2.2M) [application/x-gzip]
Saving to: ‘LKH-3.0.4.tgz’

LKH-3.0.4.tgz       100%[===================>]   2.20M   231KB/s    in 12s     

2019-11-01 16:23:33 (192 KB/s) - ‘LKH-3.0.4.tgz’ saved [2308964/2308964]

LKH-3.0.4/
LKH-3.0.4/pr2392.par
LKH-3.0.4/whizzkids96.a